In [1]:
import gsc
import numpy as np
import re
import pickle

net = gsc.load_model('../training/s50Gram_nSim70_vSim00.pkl')

In [ ]:
# test setting
input_decay_rate = 0.90
duration = 15
num_trials = 1000             # number of trials per condition

seednum = 20181107 #1  #20191230 #2   #20200127 #3  #20200110 #4  # I used the same seed number in both test cases
num_terminal_roles = net.hg.opts['max_sent_len']

# some arrays to bind to the sentence tyes for text file outputs
a1 = np.array([1,.5])
a2 = np.array([2,.5])
a3 = np.array([3,.5])
a4 = np.array([4,.5])

In [ ]:
## setup Ns Np Preamble 
input_terminals = ['Ns:0/(1,1)','Np:1/(1,2)']
partial_input_terminals = ['Ns:1/(1,1)','Np:0/(1,2)']
input_phrases = ['NPCs[2]:0/(2,1)']
input_sentence = ['S[3]:0/(3,1)']

np.random.seed(seednum)

resp_type1 = [
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##01 correct
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[3]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##02 correct + pseudo partitive
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##03 attraction + all cor NP
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##04 attraction + change to s
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##05 attraction + change s + change NP with cor loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##06 attraction + change s + change NP with wrong loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[3]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##07 attraction + pseudopartitive + pl S (grammatical)
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##08 head error, corr structure
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##09 head error, change NP loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##10 head error, change NP N with original loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##11 head error, change NP N with other loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##12 head error, no verb, change NP and S number
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##13 head error + verb, corr structure
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##14 head error + verb, change NP loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##15 head error + verb, change NP N with original loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##16 head error + verb, change NP N with other loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##17 head error + verb, change NP and S (grammatical)
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##18 local error, corr structure
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##19 local error, change NP loc (grammatical)
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##20 loc error, change NP N with original loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##21 loc error, change NP N with other loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##22 loc error, change NP loc and S
['Ns:0/(1,1)', 'Np:0/(1,2)', 'Vp:1/(1,3)', 'Vs:1/(1,4)', '*Ns:0/(2,1)', 'RC[2]:1/(2,2)', '*Vs:1/(2,3)', 'NPCs[3]:0/(3,1)', '*Vs:1/(3,2)', 'S[3]:0/(4,1)'], ##23 RC (grammatical)
['Ns:0/(1,1)', 'Np:1/(1,2)', '@:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*@:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##24 correct NP, no verb
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##25 head and loc err (grammatical)
['Ns:0/(1,1)', 'Vs:1/(1,2)', '@:1/(1,3)', '@:1/(1,4)', 'S[1]:0/(2,1)', '*@:1/(2,2)', '*@:1/(2,3)', '#:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'] ##26 head+verb (grammatical)
]

# pure 'unambiguous' external input to target non-terminal bindings in role (2,1) and (3,1)
extC_nonterminals = np.zeros(net.num_bindings)
extC_nonterminals[net.find_bindings(input_sentence + input_phrases)] = 1.

# pure 'ambiguous' external input to input-consistent terminal bindings in role (1,1) and (1,2)
# Note that I used 'input_terminals + partial_input_terimansl' 
extC_terminals = np.zeros(net.num_bindings)
extC_terminals[net.find_bindings(input_terminals + partial_input_terminals)] = 1.


res1 = np.zeros(len(resp_type1))   # will store response proportions
distGrid1 = np.zeros(1)
stateGrid1 = np.chararray((1,10))

# code follows ScIL era sims:
w = .5

    ###################################################################
    # Create an external input vector, which is a weighted sum of 
    # pure terminal input and pure non-terminal input
#print('\nweight of target terminal bindings = {:.2f}'.format(1 - w))
#print('weight of target nonterminal bindings = {:.2f}'.format(w))
curr_extC = (1-w) * extC_terminals + w * extC_nonterminals
    ###################################################################

resultStates = {}


for trialcount in range(num_trials):

    net.reset(mu=net.ep, sd=0.02)

    net.extC += 1. * curr_extC # set conceptual level input
    net.ext = net.C2N(net.extC) # rotate to neural coordinates

    for i in range(duration):
        net.run(1)
        net.extC = net.extC * input_decay_rate
        net.ext = net.C2N(net.extC)

    #count up outputs
    states = tuple(net.read_grid_point())
    
    if states in resultStates:
        resultStates[states] += 1
    else:
        resultStates[states] = 1
        
    #get traces, get grid point, get distance
    tr = net.traces['actC'][-1]
    st = net.read_grid_point()
    ref = net.get_discrete_state(st)
    dist = np.linalg.norm(ref-tr)

    #save, in separate arrays
    distGrid1 = np.append(distGrid1,dist)
    stateGrid1 = np.vstack((stateGrid1,st))
    
for key in resultStates:
    print(' '.join(list(key)), resultStates[key])
    for ri, resp1 in enumerate(resp_type1):
        if list(key) == resp1:
            res1[ri] = resultStates[key]

                
res1 /= num_trials   # from counts to probabilities
res1 = np.append(res1, 1 - res1.sum())   # add a new column containing OTHER responses



In [ ]:
## simulate Ns Ns Preamble 
input_terminals = ['Ns:0/(1,1)','Ns:1/(1,2)']
partial_input_terminals = ['Ns:1/(1,1)','Ns:0/(1,2)']
input_phrases = ['NPCs[1]:0/(2,1)']
input_sentence = ['S[3]:0/(3,1)']

np.random.seed(seednum)

resp_type2 = [
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##01 correct
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[3]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##02 correct + pseudo partitive
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##03 attraction + all cor NP
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##04 attraction + change to s
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##05 attraction + change s + change NP with cor loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##06 attraction + change s + change NP with wrong loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[3]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##07 attraction + pseudopartitive + pl S (grammatical)
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##08 head error, corr structure
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##09 head error, change NP loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##10 head error, change NP N with original loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##11 head error, change NP N with other loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##12 head error, no verb, change NP and S number
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##13 head error + verb, corr structure
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##14 head error + verb, change NP loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##15 head error + verb, change NP N with original loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##16 head error + verb, change NP N with other loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##17 head error + verb, change NP and S (grammatical)
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##18 local error, corr structure
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##19 local error, change NP loc (grammatical)
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##20 loc error, change NP N with original loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##21 loc error, change NP N with other loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##22 loc error, change NP loc and S
['Ns:0/(1,1)', 'Ns:0/(1,2)', 'Vs:1/(1,3)', 'Vs:1/(1,4)', '*Ns:0/(2,1)', 'RC[1]:1/(2,2)', '*Vs:1/(2,3)', 'NPCs[3]:0/(3,1)', '*Vs:1/(3,2)', 'S[3]:0/(4,1)'], ##23 RC (grammatical)
['Ns:0/(1,1)', 'Ns:1/(1,2)', '@:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*@:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##24 correct NP, no verb
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##25 head and loc err (grammatical)
['Ns:0/(1,1)', 'Vs:1/(1,2)', '@:1/(1,3)', '@:1/(1,4)', 'S[1]:0/(2,1)', '*@:1/(2,2)', '*@:1/(2,3)', '#:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'] ##26 head+verb (grammatical)
]

# pure 'unambiguous' external input to target non-terminal bindings in role (2,1) and (3,1)
extC_nonterminals = np.zeros(net.num_bindings)
extC_nonterminals[net.find_bindings(input_sentence + input_phrases)] = 1.

# pure 'ambiguous' external input to input-consistent terminal bindings in role (1,1) and (1,2)
# Note that I used 'input_terminals + partial_input_terimansl' 
extC_terminals = np.zeros(net.num_bindings)
extC_terminals[net.find_bindings(input_terminals + partial_input_terminals)] = 1.


res2 = np.zeros(len(resp_type2))   # will store response proportions
distGrid2 = np.zeros(1)
stateGrid2 = np.chararray((1,10))

# code follows ScIL era sims:
w = .5

    ###################################################################
    # Create an external input vector, which is a weighted sum of 
    # pure terminal input and pure non-terminal input
#print('\nweight of target terminal bindings = {:.2f}'.format(1 - w))
#print('weight of target nonterminal bindings = {:.2f}'.format(w))
curr_extC = (1-w) * extC_terminals + w * extC_nonterminals
    ###################################################################

resultStates = {}


for trialcount in range(num_trials):

    net.reset(mu=net.ep, sd=0.02)

    net.extC += 1. * curr_extC # set conceptual level input
    net.ext = net.C2N(net.extC) # rotate to neural coordinates

    for i in range(duration):
        net.run(1)
        net.extC = net.extC * input_decay_rate
        net.ext = net.C2N(net.extC)

    #count up outputs
    states = tuple(net.read_grid_point())
    
    if states in resultStates:
        resultStates[states] += 1
    else:
        resultStates[states] = 1
        
    #get traces, get grid point, get distance
    tr = net.traces['actC'][-1]
    st = net.read_grid_point()
    ref = net.get_discrete_state(st)
    dist = np.linalg.norm(ref-tr)

    #save, in separate arrays
    distGrid2 = np.append(distGrid2,dist)
    stateGrid2 = np.vstack((stateGrid2,st))
    
for key in resultStates:
    print(' '.join(list(key)), resultStates[key])
    for ri, resp1 in enumerate(resp_type2):
        if list(key) == resp1:
            res2[ri] = resultStates[key]

                
res2 /= num_trials   # from counts to probabilities
res2 = np.append(res2, 1 - res2.sum())   # add a new column containing OTHER responses



In [ ]:
# simulate Np Ns preamble 
input_terminals = ['Np:0/(1,1)','Ns:1/(1,2)']
partial_input_terminals = ['Np:1/(1,1)','Ns:0/(1,2)']
input_phrases = ['NPCp[1]:0/(2,1)']
input_sentence = ['S[4]:0/(3,1)']


np.random.seed(seednum)

resp_type3 = [
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##01 correct
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[3]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##02 correct + pseudo partitive
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##03 attraction + all cor NP
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##04 attraction + change to s
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##05 attraction + change s + change NP with cor loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##06 attraction + change s + change NP with wrong loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[3]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##07 attraction + pseudopartitive + pl S (grammatical)
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##08 head error, corr structure
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##09 head error, change NP loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##10 head error, change NP N with original loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##11 head error, change NP N with other loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##12 head error, no verb, change NP and S number
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##13 head error + verb, corr structure
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##14 head error + verb, change NP loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##15 head error + verb, change NP N with original loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##16 head error + verb, change NP N with other loc
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##17 head error + verb, change NP and S (grammatical)
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##18 local error, corr structure
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##19 local error, change NP loc (grammatical)
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##20 loc error, change NP N with original loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##21 loc error, change NP N with other loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##22 loc error, change NP loc and S
['Np:0/(1,1)', 'Ns:0/(1,2)', 'Vs:1/(1,3)', 'Vp:1/(1,4)', '*Np:0/(2,1)', 'RC[1]:1/(2,2)', '*Vp:1/(2,3)', 'NPCp[4]:0/(3,1)', '*Vp:1/(3,2)', 'S[4]:0/(4,1)'], ##23 RC (grammatical)
['Np:0/(1,1)', 'Ns:1/(1,2)', '@:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*@:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##24 correct NP, no verb
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##25 head and loc err (grammatical)
['Np:0/(1,1)', 'Vp:1/(1,2)', '@:1/(1,3)', '@:1/(1,4)', 'S[2]:0/(2,1)', '*@:1/(2,2)', '*@:1/(2,3)', '#:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'] ##26 head+verb (grammatical)
]
  
# pure 'unambiguous' external input to target non-terminal bindings in role (2,1) and (3,1)
extC_nonterminals = np.zeros(net.num_bindings)
extC_nonterminals[net.find_bindings(input_sentence + input_phrases)] = 1.

# pure 'ambiguous' external input to input-consistent terminal bindings in role (1,1) and (1,2)
# Note that I used 'input_terminals + partial_input_terimansl' 
extC_terminals = np.zeros(net.num_bindings)
extC_terminals[net.find_bindings(input_terminals + partial_input_terminals)] = 1.


res3 = np.zeros(len(resp_type3))   # will store response proportions
distGrid3 = np.zeros(1)
stateGrid3 = np.chararray((1,10))

# code follows ScIL era sims:
w = .5

    ###################################################################
    # Create an external input vector, which is a weighted sum of 
    # pure terminal input and pure non-terminal input
#print('\nweight of target terminal bindings = {:.2f}'.format(1 - w))
#print('weight of target nonterminal bindings = {:.2f}'.format(w))
curr_extC = (1-w) * extC_terminals + w * extC_nonterminals
    ###################################################################

resultStates = {}


for trialcount in range(num_trials):

    net.reset(mu=net.ep, sd=0.02)

    net.extC += 1. * curr_extC # set conceptual level input
    net.ext = net.C2N(net.extC) # rotate to neural coordinates

    for i in range(duration):
        net.run(1)
        net.extC = net.extC * input_decay_rate
        net.ext = net.C2N(net.extC)

    #count up outputs
    states = tuple(net.read_grid_point())
    
    if states in resultStates:
        resultStates[states] += 1
    else:
        resultStates[states] = 1
        
    #get traces, get grid point, get distance
    tr = net.traces['actC'][-1]
    st = net.read_grid_point()
    ref = net.get_discrete_state(st)
    dist = np.linalg.norm(ref-tr)

    #save, in separate arrays
    distGrid3 = np.append(distGrid3,dist)
    stateGrid3 = np.vstack((stateGrid3,st))
    
for key in resultStates:
    print(' '.join(list(key)), resultStates[key])
    for ri, resp1 in enumerate(resp_type3):
        if list(key) == resp1:
            res3[ri] = resultStates[key]

                
res3 /= num_trials   # from counts to probabilities
res3 = np.append(res3, 1 - res3.sum())   # add a new column containing OTHER responses



In [ ]:
# simulate Np Np preamble 
input_terminals = ['Np:0/(1,1)','Np:1/(1,2)']
partial_input_terminals = ['Np:1/(1,1)','Np:0/(1,2)']
input_phrases = ['NPCp[2]:0/(2,1)']
input_sentence = ['S[4]:0/(3,1)']


np.random.seed(seednum)

resp_type4 = [
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##01 correct
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[3]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##02 correct + pseudo partitive
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##03 attraction + all cor NP
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##04 attraction + change to s
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##05 attraction + change s + change NP with cor loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##06 attraction + change s + change NP with wrong loc
['Np:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[3]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##07 attraction + pseudopartitive + pl S (grammatical)
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##08 head error, corr structure
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##09 head error, change NP loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##10 head error, change NP N with original loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##11 head error, change NP N with other loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##12 head error, no verb, change NP and S number
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##13 head error + verb, corr structure
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##14 head error + verb, change NP loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##15 head error + verb, change NP N with original loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##16 head error + verb, change NP N with other loc
['Ns:0/(1,1)', 'Np:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##17 head error + verb, change NP and S (grammatical)
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##18 local error, corr structure
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##19 local error, change NP loc (grammatical)
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[2]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##20 loc error, change NP N with original loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##21 loc error, change NP N with other loc
['Np:0/(1,1)', 'Ns:1/(1,2)', 'Vp:1/(1,3)', '@:1/(1,4)', 'NPCp[1]:0/(2,1)', '*Vp:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##22 loc error, change NP loc and S
['Np:0/(1,1)', 'Np:0/(1,2)', 'Vp:1/(1,3)', 'Vp:1/(1,4)', '*Np:0/(2,1)', 'RC[2]:1/(2,2)', '*Vp:1/(2,3)', 'NPCp[4]:0/(3,1)', '*Vp:1/(3,2)', 'S[4]:0/(4,1)'], ##23 RC (grammatical)
['Np:0/(1,1)', 'Np:1/(1,2)', '@:1/(1,3)', '@:1/(1,4)', 'NPCp[2]:0/(2,1)', '*@:1/(2,2)', '*@:1/(2,3)', 'S[4]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##24 correct NP, no verb
['Ns:0/(1,1)', 'Ns:1/(1,2)', 'Vs:1/(1,3)', '@:1/(1,4)', 'NPCs[1]:0/(2,1)', '*Vs:1/(2,2)', '*@:1/(2,3)', 'S[3]:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'], ##25 head and loc err (grammatical)
['Np:0/(1,1)', 'Vp:1/(1,2)', '@:1/(1,3)', '@:1/(1,4)', 'S[2]:0/(2,1)', '*@:1/(2,2)', '*@:1/(2,3)', '#:0/(3,1)', '*@:1/(3,2)', '#:0/(4,1)'] ##26 head+verb (grammatical)
]

# pure 'unambiguous' external input to target non-terminal bindings in role (2,1) and (3,1)
extC_nonterminals = np.zeros(net.num_bindings)
extC_nonterminals[net.find_bindings(input_sentence + input_phrases)] = 1.

# pure 'ambiguous' external input to input-consistent terminal bindings in role (1,1) and (1,2)
# Note that I used 'input_terminals + partial_input_terimansl' 
extC_terminals = np.zeros(net.num_bindings)
extC_terminals[net.find_bindings(input_terminals + partial_input_terminals)] = 1.


res4 = np.zeros(len(resp_type4))   # will store response proportions
distGrid4 = np.zeros(1)
stateGrid4 = np.chararray((1,10))

# code follows ScIL era sims:
w = .5

    ###################################################################
    # Create an external input vector, which is a weighted sum of 
    # pure terminal input and pure non-terminal input
#print('\nweight of target terminal bindings = {:.2f}'.format(1 - w))
#print('weight of target nonterminal bindings = {:.2f}'.format(w))
curr_extC = (1-w) * extC_terminals + w * extC_nonterminals
    ###################################################################

resultStates = {}


for trialcount in range(num_trials):

    net.reset(mu=net.ep, sd=0.02)

    net.extC += 1. * curr_extC # set conceptual level input
    net.ext = net.C2N(net.extC) # rotate to neural coordinates

    for i in range(duration):
        net.run(1)
        net.extC = net.extC * input_decay_rate
        net.ext = net.C2N(net.extC)

    #count up outputs
    states = tuple(net.read_grid_point())
    
    if states in resultStates:
        resultStates[states] += 1
    else:
        resultStates[states] = 1
        
    #get traces, get grid point, get distance
    tr = net.traces['actC'][-1]
    st = net.read_grid_point()
    ref = net.get_discrete_state(st)
    dist = np.linalg.norm(ref-tr)

    #save, in separate arrays
    distGrid4 = np.append(distGrid4,dist)
    stateGrid4 = np.vstack((stateGrid4,st))
    
for key in resultStates:
    print(' '.join(list(key)), resultStates[key])
    for ri, resp1 in enumerate(resp_type4):
        if list(key) == resp1:
            res4[ri] = resultStates[key]

                
res4 /= num_trials   # from counts to probabilities
res4 = np.append(res4, 1 - res4.sum())   # add a new column containing OTHER responses




In [ ]:
print(res4)

In [ ]:
## save outputs: states & distances, and averaged results
r1 = np.append(a1,res1,axis=0)
r2 = np.append(a2,res2,axis=0)
r3 = np.append(a3,res3,axis=0)
r4 = np.append(a4,res4,axis=0)

r = np.stack((r1,r2,r3,r4),axis=0)

np.savetxt('s50n70v00s50_r1_states_av.csv',r,fmt='%d \t %.1f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t%.4f')

g1 = np.c_[stateGrid1,distGrid1]
g2 = np.c_[stateGrid2,distGrid2]
g3 = np.c_[stateGrid3,distGrid3]
g4 = np.c_[stateGrid4,distGrid4]

g1 = np.delete(g1,0,0)
g2 = np.delete(g2,0,0)
g3 = np.delete(g3,0,0)
g4 = np.delete(g4,0,0)

aa1 = np.tile(a1,(num_trials,1))
aa2 = np.tile(a2,(num_trials,1))
aa3 = np.tile(a3,(num_trials,1))
aa4 = np.tile(a4,(num_trials,1))

g = np.vstack((g1,g2,g3,g4))
a = np.vstack((aa1,aa2,aa3,aa4))
g = np.hstack((a,g))

np.savetxt('s50n70v00s50_r1_states_all.csv',g,fmt='%s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s ')

In [ ]:
print(r)